In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## The forward and backward passes

In [2]:
from exp.nb_01 import *

In [3]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        (x_train, y_train), (x_valid, y_valid), _ = pickle.load(f, encoding='latin-1')
    return map(tensor, [x_train, y_train, x_valid, y_valid])
        
def normalize(x, m, s): return (x - m) / s

In [4]:
x_train, y_train, x_valid, y_valid = get_data()

In [5]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [6]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
x_train.mean(), x_train.std()

(tensor(0.0001), tensor(1.))

In [8]:
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f"Near zero: {a}"

In [9]:
test_near_zero(x_train.mean())

In [10]:
test_near_zero(1 - x_train.std())

In [11]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [72]:
nh = 50

In [73]:
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [74]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1/math.sqrt(m))

In [75]:
x_valid.mean(), x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [76]:
def lin(x, w, b): return x @ w + b

In [77]:
t = lin(x_valid, w1, b1)

In [78]:
# `t`, by design, ought to have mean 0 and std 1. Why?  
t.mean(), t.std()

(tensor(0.0274), tensor(1.0227))

In [79]:
def relu(x): return x.clamp_min(0.)

In [80]:
t = relu(lin(x_valid, w1, b1))

In [81]:
t.mean(), t.std()

(tensor(0.4199), tensor(0.6179))

In [82]:
w1 = torch.randn(m, nh) * math.sqrt(2 / m)

In [83]:
w1.mean(), w1.std()

(tensor(-6.1446e-05), tensor(0.0507))

In [84]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.5628), tensor(0.8480))

In [85]:
from torch.nn import init

In [86]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [87]:
w1.mean(), w1.std()

(tensor(0.0001), tensor(0.0504))

In [88]:
t.mean(), t.std()

(tensor(0.5245), tensor(0.7845))

In [89]:
w1.shape

torch.Size([784, 50])

In [90]:
import torch.nn

In [31]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

In [33]:
torch.nn.Linear.forward??

In [34]:
torch.nn.functional.linear??

In [34]:
torch.nn.Conv2d??

In [91]:
def relu(x): return x.clamp_min(0.) - .5

In [92]:
w1 = torch.randn(m, nh) * math.sqrt(2/m)
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.0355), tensor(0.8214))

In [93]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [35]:
%%timeit -n 10

_ = model(x_valid)

4.73 ms ± 2.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

### Loss function: MSE

In [94]:
model(x_valid).shape

torch.Size([10000, 1])

In [95]:
y_valid.shape

torch.Size([10000])

In [96]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [97]:
y_train, y_valid = y_train.float(), y_valid.float()

In [98]:
preds = model(x_train)

In [99]:
mse(preds, y_train)

tensor(27.9905)

### Gradients and backward pass

In [100]:
def mse_grad(inp, targ):
    '''
    dLdYhat, or gradient of loss w.r.t model output.
    '''
    inp.g =  2 * (inp - targ[:,None]) / inp.shape[0]

In [101]:
def lin_grad(inp, out, w, b):
    '''
    Forward pass: out = inp . w + b
    Want: dLdinp, dLdw, dLdb
    '''
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(dim=0)

In [102]:
def relu_grad(inp, out):
    '''
    A = g(Z) = ReLU(Z)
    dLdZ = dLdA . dAdZ
    The gradient of the loss w.r.t inputs to ReLu(). 
    '''
    inp.g = out.g * (inp > 0).float()

In [103]:
def forward_and_backward(inp, targ):
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    loss = mse(out, targ)
    
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [104]:
forward_and_backward(x_train, y_train)

In [105]:
w1g = w1.g.clone()
b1g = b1.g.clone()
w2g = w2.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

We cheat a little bit and PyTorch autograd to check our results.

In [106]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [107]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [108]:
loss = forward(xt2, y_train)

In [109]:
loss.backward()

In [110]:
test_near(w22.grad, w2.g)
test_near(b22.grad, b2.g)
test_near(w12.grad, w1.g)
test_near(b12.grad, b1.g)
test_near(xt2.grad, x_train.g)

In [111]:
near??

## Refactor model

### Layers as classes

In [159]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - .5
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g * (self.inp > 0.).float()

In [160]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.out.g[:,None,:] * self.inp[:,:,None]).sum(dim=0)
        self.b.g = self.out.g.sum(dim=0)

In [161]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ[:,None]
        self.out = (inp - self.targ).pow(2).mean(dim=0)
        return self.out
    
    def backward(self):
        self.inp.g = 2 * (self.inp - self.targ) / self.inp.shape[0]

In [162]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
            
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [163]:
w1.g, b1.g, w2.g, b2.g = 4 * [None]
model = Model(w1, b1, w2, b2)

In [164]:
%%time 

loss = model(x_train, y_train)

CPU times: user 128 ms, sys: 12 ms, total: 140 ms
Wall time: 37.7 ms


In [118]:
%%time

model.backward()

CPU times: user 3.7 s, sys: 3.61 s, total: 7.31 s
Wall time: 6.36 s


In [119]:
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(ig, x_train.g)

### Module.forward()

In [120]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')    
    def backward(self): self.bwd(self.out, *self.args)

In [121]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.) - .5
    def bwd(self, out, inp): inp.g = out.g * (inp > 0).float()

In [139]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
    def forward(self, inp): return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(dim=0)

In [140]:
class Mse(Module):
    def forward(self, inp, targ): return (inp - targ[:,None]).pow(2).mean(dim=0)
    def bwd(self, out, inp, targ): inp.g = 2 * (inp - targ[:,None]) / inp.shape[0]

In [141]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [142]:
w1.g, b1.g, w2.g, b2.g = 4 * [None]
model = Model()

In [143]:
%%time 

loss = model(x_train, y_train)

CPU times: user 128 ms, sys: 12 ms, total: 140 ms
Wall time: 39.9 ms


In [144]:
%%time 

model.backward()

CPU times: user 504 ms, sys: 76 ms, total: 580 ms
Wall time: 149 ms


In [145]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [146]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
    def forward(self, inp): return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(dim=0)

In [147]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [148]:
%%time

loss = model(x_train, y_train)

CPU times: user 140 ms, sys: 8 ms, total: 148 ms
Wall time: 40.9 ms


In [149]:
%%time

model.backward()

CPU times: user 524 ms, sys: 80 ms, total: 604 ms
Wall time: 158 ms


In [150]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [152]:
from torch import nn

In [154]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [155]:
model = Model(m, nh, 1)

In [156]:
%%time 

loss = model(x_train, y_train)

CPU times: user 136 ms, sys: 16 ms, total: 152 ms
Wall time: 65.4 ms


In [157]:
%%time

loss.backward()

CPU times: user 128 ms, sys: 12 ms, total: 140 ms
Wall time: 102 ms


# fin